## Integrantes

---
* Karina Barrientos
* kbarrientosb1@correo.uss.cl
* 21.460.339-K
---

* Lucas Brown Ibieta
* lbrowni@correo.uss.cl
* 21.063.252-2





# Solemne 3: Prediccion de satisfaccion de servicio en base a hábitos de compra

Usted ha sido recientemente contratado como científico de datos en una tienda de venta de ropa online. Dado el gran volumen de ventas que tiene la tienda los asistentes de la empresa trabajan sin descanso ayudando a los clientes con el proceso de compra. Dado que recientemente se han empezado a registrar esperas muy largas para ser atendido por uno de los asistentes, a alguien en la junta directiva se le ocurrio que se podia ahorrar tiempo si los asistentes no realizaran una “breve” encuesta de satisfaccion a cada cliente. Esta encuensta se traduce internamente en un puntaje único, lo que ayudaria a predecirlo  a partir de datos de los clientes recogidos de manera automática, y si es cierto, a disminuir los tiempos de espera de los clientes por un asistente sin tener que contrar mas personas para ese cargo.

El archivo dataset3.csv contiene un dataset con los resultados de información recogida de manera automática (17 variables) para 3900 clientes.

*Con esta información, determine si es posible predecir de manera robusta un predictor de puntaje de satissfacionen a partir de otra información que se recoge de manera automática para los clientes .*

Deben asegurarse de que sus sistema generaliza  y que han usado el mejor algoritmo regrersor con los mejores parametros posibles para reolver esta tarea. Documenten el proceso y explique cada una de las decisiones que toma y que criterios las sustentan.

# Tarea:
Aprovechando la gran cantidad de datos existentes en el dataset, construya un sistema que a través de  algoritmos de regresión nos permita establecer la viabilidad de predecir el puntaje de satisfaccion a paritr de otra data de facil recoleccion.

El procedimiento a seguir debe ser el siguiente:

1. Utilicen al menos 3 algoritmos de los explicados en la unidad 3 con distintos parametros y los 17 descriptores del dataset. Asegurense de las capacidades de generalización de cada algoritmo/set de parametros y comparen el desempeño de todos ellos.

2. Usen al menos un metodo para seleccionar los mejores 10 descriptores y compare el desempeño con las versiones de los mismos algoritmos del punto anteriors. Expliquen la diferencia en desempeños

3. Estudien ahora el efecto de normalizar los datos. ¿Los parametros optimos y el desempeño son distintos?, expliquen las diferencias observadas
       
Se entrega el jupyter notebook. Use celdas con texto para justificar y explicar cada paso. Asegurense de que la primera celda contiene el Nombre completo, email y rut de los miembros del equipo.

![alt text](Untitled.png)

## **1. Importación de Librerías y Carga de Datos**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_regression

df = pd.read_csv('dataset3.csv')
print("¡Archivo 'dataset3.csv' cargado exitosamente!")

Este bloque es el punto de entrada. Importamos las librerías necesarias de sklearn y pandas. La lógica principal intenta leer dataset3.csv directamente del disco.

In [2]:
print("COLUMNAS:", df.columns.tolist()); print("\nPREVIEW DATOS:"); display(df.head())

COLUMNAS: ['Review Rating', 'Age', 'Gender', 'Item Purchased', 'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season', 'Subscription Status', 'Payment Method', 'Shipping Type', 'Shipping Type.1', 'Discount Applied', 'Promo Code Used', 'Previous Purchases', 'Preferred Payment Method', 'Frequency of Purchases']

PREVIEW DATOS:


,Review Rating,Age,Gender,Item Purchased,Purchase Amount (USD),Location,Size,Color,Season,Subscription Status,Payment Method,Shipping Type,Shipping Type.1,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,3.1,55,Male,Blouse,53,Kentucky,L,Gray,Winter,Yes,Credit Card,Express,Express,Yes,Yes,14,Venmo,Fortnightly
1,3.1,19,Male,Sweater,64,Maine,L,Maroon,Winter,Yes,Bank Transfer,Express,Express,Yes,Yes,2,Cash,Fortnightly
2,3.1,50,Male,Jeans,73,Massachusetts,S,Maroon,Spring,Yes,Cash,Free_Shipping,Free_Shipping,Yes,Yes,23,Credit Card,Weekly
3,3.5,21,Male,Sandals,90,Rhode Island,M,Maroon,Spring,Yes,PayPal,Next_Day_Air,Next_Day_Air,Yes,Yes,49,PayPal,Weekly
4,2.7,45,Male,Blouse,49,Oregon,M,Turquoise,Spring,Yes,Cash,Free_Shipping,Free_Shipping,Yes,Yes,31,PayPal,Annually


In [ ]:
try:   
# 2. DEFINICIÓN DE TARGET Y FEATURES
    target_col = 'Review Rating'
    
    # Separamos X (Variables) e y (Target)
    y = df[target_col]
    X_raw = df.drop(target_col, axis=1)

    # 3. PREPROCESAMIENTO
    print("Convirtiendo variables de texto (categóricas) a numéricas...")
    X = pd.get_dummies(X_raw, drop_first=True)
    
    print(f"Dimensiones después de transformar texto a números: {X.shape}")
    print("Ejemplo de nuevas columnas generadas:", list(X.columns[:5]))

except FileNotFoundError:
    print("ERROR: No se encuentra 'dataset3.csv'. Asegúrate de subirlo o ponerlo en la carpeta.")


Se separa automáticamente las variables independientes ($X$) de la variable objetivo ($y$), asumiendo que el objetivo está en la última columna.

## **2. Preparación de Datos (Train/Test Split)**

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,  # 20% para test
    random_state=42
)

print(f"Datos de Entrenamiento: {X_train.shape[0]} muestras")
print(f"Datos de Prueba: {X_test.shape[0]} muestras")

# Inicializamos una lista para guardar los resultados de cada experimento
resultados = []

Aquí dividimos los datos en dos conjuntos: Entrenamiento (80%) y Prueba (20%). Esta separación es muy importante para asegurarse de que el sistema generalize bien. Si evaluáramos el modelo con los mismos datos que usamos para entrenarlo, obtendríamos resultados engañosos ya que solo mira los datos que ya conoce (overfitting). Los datos de prueba actúan como clientes "nuevos" que el sistema nunca ha visto.

## **3. Definición de Función de Evaluación**

In [ ]:
def evaluar_modelo(nombre, modelo, params_desc):
    """
    Entrena el modelo proporcionado y evalúa su desempeño tanto en 
    el set de entrenamiento como en el de prueba.
    """
    # Entrenamiento
    modelo.fit(X_train, y_train)
    
    # Predicciones
    y_train_pred = modelo.predict(X_train)
    y_test_pred = modelo.predict(X_test)
    
    # Cálculo de métricas
    # MSE (Error Cuadrático Medio) - Mide el error promedio al cuadrado
    mse_test = mean_squared_error(y_test, y_test_pred)
    
    # R2 Score - Mide qué tan bien se ajusta el modelo (1.0 es perfecto)
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    # Guardar métricas en la lista de resultados
    resultados.append({
        'Algoritmo': nombre,
        'Parametros': params_desc,
        'MSE Test': mse_test,
        'R2 Train': r2_train,
        'R2 Test': r2_test,
        'Generalizacion (Gap)': abs(r2_train - r2_test) 
    })
    
    print(f"Modelo: {nombre} ({params_desc})")
    print(f"   -> R2 Train: {r2_train:.4f} | R2 Test: {r2_test:.4f}")
    print(f"   -> Gap Generalización: {abs(r2_train - r2_test):.4f}")
    print("-" * 30)

Definimos una función estándar para evitar repetir código. Esta función toma cualquier modelo (KNN, Árbol, SVR), lo entrena con tus datos y calcula métricas clave:

$R^2$ (Coeficiente de determinación): Nos dice el porcentaje de variabilidad de la satisfacción que el modelo puede explicar.

Gap de Generalización: La diferencia entre el rendimiento en entrenamiento y prueba. Si es muy grande, hay overfitting.